In [ ]:
from csidhtools import CSIDHCW
import chipwhisperer as cw
from tqdm import tqdm, trange
import random

In [ ]:
if 'csidh' in globals() or 'csidh' in locals():
    csidh.reset_target()
    csidh.dis()

In [ ]:
csidh = CSIDHCW(PATH + '/csidh-target/src/', attack_type=attack_type)

In [ ]:
csidh.setup()

In [ ]:
csidh.flash_target()
csidh.reset_target()

In [ ]:
csidh.reset_target()
csidh.scope.arm()
csidh.action()
ret = csidh.scope.capture()
if ret:
    print("Timeout happened during acquisition")
PUBLIC_EXPECTED = csidh.public_with_errors
MAX_EXT_OFFSET = csidh.scope.adc.trig_count

In [ ]:
print("#" * 80)
print("First run of CSIDH")
print("#" * 80)
print(f"PUBLIC: {PUBLIC_EXPECTED}")
print(f"TRIGS:  {MAX_EXT_OFFSET}")
print("#" * 80)

In [ ]:
from csidhtools.search import Unit, generate_population, write_cache_to_file
from collections import OrderedDict
import numpy as np
cache = OrderedDict()


def evaluate_unit(csidh, unit, num_measurements=1):
    """Evaluates a single unit"""
    csidh.scope.glitch.num_glitches = 1
    if csidh.scope._is_husky:
        csidh.scope.glitch.width = int(unit.width)
        csidh.scope.glitch.offset = int(unit.offset)
        csidh.scope.glitch.repeat = int(unit.repeat)
        csidh.scope.glitch.ext_offset = int(unit.ext_offset)
    else:
        csidh.scope.glitch.width = unit.width
        csidh.scope.glitch.offset = unit.offset
        csidh.scope.glitch.repeat = unit.repeat
        csidh.scope.glitch.ext_offset = unit.ext_offset

    # Perform the measurements
    measurements = []
    responses = []

    for _ in range(num_measurements):
        csidh.reset_target()
        csidh.scope.glitch.state = None
        csidh.scope.arm()
        ret = csidh.action()
        csidh.scope.io.vglitch_reset()
        if ret:
            logging.error("Timeout happened during acquisition")


        public_received = csidh.public_with_errors
        if not isinstance(public_received, int):
            measurements.append("RESET")
        elif public_received == PUBLIC_EXPECTED:
            measurements.append("NORMAL")
        else:
            measurements.append("JUSTRIGHT")
            responses.append(public_received)

    unit.width = csidh.scope.glitch.width  # CW rounds the values
    unit.offset = csidh.scope.glitch.offset
    unit.repeat = csidh.scope.glitch.repeat
    unit.measurements = measurements
    unit.responses = responses
    

    # Classify
    if not all(m == measurements[0] for m in measurements):
        unit.type = "CHANGING"
        N_normal = sum(1 for m in measurements if m == "NORMAL")
        N_reset = sum(1 for m in measurements if m == "RESET")
        N_justright = sum(1 for m in measurements if m == "JUSTRIGHT")
        unit.fitness = 4 + 1.2 * N_justright + 0.2 * N_normal + 0.5 * N_reset
    else:
        if measurements[0] == "NORMAL":
            unit.type = "NORMAL"
            unit.fitness = 2
        elif measurements[0] == "RESET":
            unit.type = "RESET"
            unit.fitness = 5
        elif measurements[0] == "JUSTRIGHT":
            unit.type = "JUSTRIGHT"
            unit.fitness = 10
    cache[unit] = unit.fitness

def evaluate_batch(csidh, population, evaluate_unit=evaluate_unit):
    uncached = [u for u in population if u not in cache]
    if not uncached:
        return [u.fitness for u in population]

    to_visit = np.array(uncached)

    for unit in population:
        if unit in cache:
            unit.fitness = cache[unit]

    for unit in to_visit:
        evaluate_unit(csidh, unit)

    return [u.fitness for u in population]

In [ ]:
def generate_whole_population():
    population = []
    for offset in range(Unit.OFFSET_MIN, Unit.OFFSET_MAX+1):
        for width in range(Unit.WIDTH_MIN, Unit.WIDTH_MAX+1):
            for ext_offset in range(Unit.EXT_OFFSET_MIN, Unit.EXT_OFFSET_MAX+1):
                for repeat in range(Unit.REPEAT_MIN, Unit.REPEAT_MAX+1, 1):
                    unit = Unit(repr=f"{ext_offset},{offset},{width},{repeat},None,0")
                    population.append(unit)
    return population